# Teams Clone - API Testing & Exploration

Interactive notebook for testing all API endpoints and exploring the backend.

## Available APIs:
- 🔐 Authentication (Login/Register)
- 💬 Messages (Send/Receive/React)
- 📞 Calls (Create/Join/Leave)
- 🤖 RL Environment (Reset/Step/State)
- 📅 Calendar (Events/Schedule)

## 1. Setup

## ✅ Verified Working Endpoints

Based on backend inspection, these endpoints are confirmed working:

### Authentication (No token required)
- `POST /auth/register` - Register new user
- `POST /auth/login` - Login user

### Calls API (No token required)
- `GET /calls` - Get all active calls
- `GET /calls/:callId` - Get specific call
- `POST /calls/create` - Create new call
- `POST /calls/:callId/join` - Join a call
- `POST /calls/:callId/leave` - Leave a call

### RL Environment (No token required)
- `GET /env/actions` - Get available actions
- `GET /env/state` - Get current state
- `POST /env/reset` - Reset environment
- `POST /env/step` - Take an action
- `GET /env/stats` - Get statistics
- `GET /env/history` - Get episode history

### Messages API (Requires token)
- `POST /api/messages` - Send message (requires sender_email, receiver_email, message)
- `GET /api/messages/:senderEmail/:receiverEmail` - Get conversation

### Calendar API
- `POST /calendar/events` - Create event
- `GET /calendar/events` - Get all events

### User API (Requires token)
- `GET /user/contacts/:email` - Get contacts
- `GET /user/channels/:email` - Get channels

In [1]:
import requests
import json
from pprint import pprint

# Configuration
BASE_URL = "http://localhost:3001"
headers = {'Content-Type': 'application/json'}

# Helper function
def test_endpoint(method, endpoint, data=None, description=""):
    """Test an API endpoint and display results"""
    print("=" * 60)
    print(f"🔍 Testing: {description}")
    print(f"📍 {method} {endpoint}")
    print("=" * 60)
    
    try:
        url = f"{BASE_URL}{endpoint}"
        
        if method == "GET":
            response = requests.get(url, headers=headers)
        elif method == "POST":
            response = requests.post(url, json=data, headers=headers)
        elif method == "PUT":
            response = requests.put(url, json=data, headers=headers)
        elif method == "DELETE":
            response = requests.delete(url, headers=headers)
        
        print(f"\n✅ Status Code: {response.status_code}")
        print(f"\n📦 Response:")
        pprint(response.json())
        
        return response.json()
    except Exception as e:
        print(f"\n❌ Error: {e}")
        return None

print("✅ Setup complete!")

✅ Setup complete!


## 2. Authentication Tests

In [ ]:
# Register a new user
import time
register_data = {
    "name": "Test User",
    "email": f"test{int(time.time())}@example.com",  # Unique email
    "password": "password123"
}

result = test_endpoint(
    "POST",
    "/auth/register",  # FIXED: Changed from /user/register to /auth/register
    register_data,
    "Register New User"
)

# Save user email for login
if result and 'token' in result:
    USER_EMAIL = register_data['email']
    print(f"\n📧 User email saved: {USER_EMAIL}")

🔍 Testing: Register New User
📍 POST /user/register

✅ Status Code: 401

📦 Response:
{'error': 'No token provided'}


In [ ]:
# Login
# Use the email from registration or a default
if 'USER_EMAIL' not in globals():
    USER_EMAIL = "test@example.com"

login_data = {
    "email": USER_EMAIL,
    "password": "password123"
}

result = test_endpoint(
    "POST",
    "/auth/login",  # FIXED: Changed from /user/login to /auth/login
    login_data,
    "User Login"
)

# Save token for future requests
if result and 'token' in result:
    TOKEN = result['token']
    headers['Authorization'] = f'Bearer {TOKEN}'
    print(f"\n🔑 Token saved: {TOKEN[:30]}...")

🔍 Testing: User Login
📍 POST /user/login

✅ Status Code: 401

📦 Response:
{'error': 'No token provided'}


## 3. Message API Tests

In [ ]:
# Send a message
# Note: This endpoint expects sender_email, receiver_email, and message
message_data = {
    "sender_email": USER_EMAIL if 'USER_EMAIL' in globals() else "test@example.com",
    "receiver_email": "admin@example.com",  # Or any other user email
    "message": "Hello from API test notebook! 🚀"
}

result = test_endpoint(
    "POST",
    "/api/messages",  # This endpoint uses a different structure than expected
    message_data,
    "Send Message"
)

# Note: To use this endpoint, both users must exist in the database

🔍 Testing: Send Message
📍 POST /api/messages

✅ Status Code: 400

📦 Response:
{'error': 'Missing required fields', 'success': False}


In [ ]:
# Get messages between two users
# Note: The messages endpoint structure is /api/messages/:senderEmail/:receiverEmail
sender_email = USER_EMAIL if 'USER_EMAIL' in globals() else "test@example.com"
receiver_email = "admin@example.com"

result = test_endpoint(
    "GET",
    f"/api/messages/{sender_email}/{receiver_email}",  # FIXED: Correct endpoint structure
    description="Get Messages Between Users"
)

# Alternative: If you want to test other working endpoints, try:
# - GET /calls (for active calls)
# - GET /env/actions (for RL environment actions)
# - GET /env/state (for RL environment state)

🔍 Testing: Get Messages from 'general' channel
📍 GET /api/messages/general

✅ Status Code: 404

📦 Response:

❌ Error: Expecting value: line 1 column 1 (char 0)


## 4. Call API Tests

In [ ]:
# Create a new call
call_data = {
    "type": "video",
    "channelId": "test-channel-123",
    "userId": "user-test-001",
    "userName": "Test User",
    "participants": []
}

result = test_endpoint(
    "POST",
    "/calls/create",
    call_data,
    "Create Video Call"
)

# Save call ID
if result and 'call' in result:
    CALL_ID = result['call']['id']
    print(f"\n📞 Call ID saved: {CALL_ID}")

In [ ]:
# Get all active calls
result = test_endpoint(
    "GET",
    "/calls",
    description="Get All Active Calls"
)

In [ ]:
# Get specific call details
if 'CALL_ID' in globals():
    result = test_endpoint(
        "GET",
        f"/calls/{CALL_ID}",
        description="Get Call Details"
    )

In [ ]:
# Join the call
if 'CALL_ID' in globals():
    join_data = {
        "userId": "user-test-002",
        "userName": "Test User 2"
    }
    
    result = test_endpoint(
        "POST",
        f"/calls/{CALL_ID}/join",
        join_data,
        "Join Call"
    )

In [ ]:
# Leave the call
if 'CALL_ID' in globals():
    leave_data = {
        "userId": "user-test-002"
    }
    
    result = test_endpoint(
        "POST",
        f"/calls/{CALL_ID}/leave",
        leave_data,
        "Leave Call"
    )

## 5. RL Environment Tests

In [ ]:
# Reset environment (start new episode)
reset_data = {
    "taskType": "greeting_response"
}

result = test_endpoint(
    "POST",
    "/env/reset",
    reset_data,
    "Reset RL Environment"
)

# Save episode ID
if result and 'episodeId' in result:
    EPISODE_ID = result['episodeId']
    print(f"\n🎮 Episode ID: {EPISODE_ID}")

In [ ]:
# Get current state
result = test_endpoint(
    "GET",
    "/env/state",
    description="Get Current Environment State"
)

In [ ]:
# Get available actions
result = test_endpoint(
    "GET",
    "/env/actions",
    description="Get Available Actions"
)

In [ ]:
# Take an action
action_data = {
    "action": {
        "type": "send_message",
        "payload": {
            "content": "Hello! This is from the API test notebook."
        }
    }
}

result = test_endpoint(
    "POST",
    "/env/step",
    action_data,
    "Execute Action (Send Message)"
)

In [ ]:
# Get episode statistics
result = test_endpoint(
    "GET",
    "/env/stats",
    description="Get Episode Statistics"
)

In [ ]:
# Get episode history
result = test_endpoint(
    "GET",
    "/env/history?limit=10",
    description="Get Last 10 Episodes"
)

## 6. Calendar API Tests

In [ ]:
# Create calendar event
event_data = {
    "title": "Team Meeting",
    "description": "Discuss project progress",
    "startTime": "2025-11-04T10:00:00Z",
    "endTime": "2025-11-04T11:00:00Z",
    "participants": ["user1", "user2"]
}

result = test_endpoint(
    "POST",
    "/calendar/events",
    event_data,
    "Create Calendar Event"
)

In [ ]:
# Get all events
result = test_endpoint(
    "GET",
    "/calendar/events",
    description="Get All Calendar Events"
)

## 7. Batch Testing

In [ ]:
# Test all working endpoints in sequence
test_suite = [
    ("GET", "/env/actions", None, "Available RL Actions"),
    ("GET", "/env/state", None, "Current RL State"),
    ("GET", "/calls", None, "Active Calls"),
    ("GET", "/health", None, "Health Check"),
]

print("\n" + "=" * 60)
print("🚀 Running Batch Tests")
print("=" * 60 + "\n")

results = []
for method, endpoint, data, desc in test_suite:
    result = test_endpoint(method, endpoint, data, desc)
    results.append({
        'endpoint': endpoint,
        'success': result is not None
    })
    print("\n")

# Summary
print("\n" + "=" * 60)
print("📊 Test Summary")
print("=" * 60)
success_count = sum(1 for r in results if r['success'])
print(f"Passed: {success_count}/{len(results)}")
print(f"Success Rate: {(success_count/len(results)*100):.1f}%")

## 8. Custom Request Builder

In [ ]:
# Build and send a custom request
custom_method = "GET"  # Change to GET, POST, PUT, DELETE
custom_endpoint = "/env/state"  # Change to any endpoint
custom_data = {}  # Add request body if needed
custom_description = "Custom Request"

result = test_endpoint(
    custom_method,
    custom_endpoint,
    custom_data if custom_data else None,
    custom_description
)

## 9. Quick Testing Guide

### ✅ Endpoints that work WITHOUT authentication:
1. **GET /health** - Health check
2. **GET /env/actions** - Get RL actions
3. **GET /env/state** - Get RL state
4. **POST /env/reset** - Reset RL environment
5. **POST /env/step** - Take RL action
6. **GET /calls** - Get active calls
7. **POST /calls/create** - Create a call
8. **POST /auth/register** - Register user
9. **POST /auth/login** - Login user

### 🔐 Endpoints that REQUIRE authentication (Bearer token):
1. **GET /user/contacts/:email** - Get user contacts
2. **GET /user/channels/:email** - Get user channels
3. **POST /api/messages** - Send message
4. **GET /api/messages/:sender/:receiver** - Get messages

### 📝 Example Quick Tests:
```python
# Test 1: Health Check
test_endpoint("GET", "/health", None, "Health Check")

# Test 2: Get RL Actions
test_endpoint("GET", "/env/actions", None, "RL Actions")

# Test 3: Get Active Calls
test_endpoint("GET", "/calls", None, "Active Calls")

# Test 4: Register User
test_endpoint("POST", "/auth/register", {
    "name": "New User",
    "email": "newuser@example.com",
    "password": "password123"
}, "Register")

# Test 5: Create Call
test_endpoint("POST", "/calls/create", {
    "type": "video",
    "channelId": "test-channel",
    "userId": "user-123",
    "userName": "Test User"
}, "Create Call")
```